<a href="https://colab.research.google.com/github/thiagorocha14/integracao-numerica/blob/master/Jogo_da_Velha_e_redes_neurais.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

JOGO DA VELHA

In [ ]:
# @title Jogo para criar dados de treinamento (.CSV)
import csv

In [5]:
def criar_tabuleiro():
    return [" " for _ in range(9)]

In [4]:
def mostrar_tabuleiro(tabuleiro):
    print(f"{tabuleiro[0]} | {tabuleiro[1]} | {tabuleiro[2]}")
    print("--+---+--")
    print(f"{tabuleiro[3]} | {tabuleiro[4]} | {tabuleiro[5]}")
    print("--+---+--")
    print(f"{tabuleiro[6]} | {tabuleiro[7]} | {tabuleiro[8]}")

In [3]:
def verificar_vitoria(tabuleiro, jogador):
    combinacoes_vencedoras = [
        [0, 1, 2], [3, 4, 5], [6, 7, 8],  # Linhas
        [0, 3, 6], [1, 4, 7], [2, 5, 8],  # Colunas
        [0, 4, 8], [2, 4, 6]              # Diagonais
    ]
    return any(all(tabuleiro[pos] == jogador for pos in combinacao) for combinacao in combinacoes_vencedoras)

In [6]:
def verificar_empate(tabuleiro):
    return " " not in tabuleiro

In [12]:
def salvar_resultados_csv(resultados):
    import csv
    with open("resultados.csv", mode="a", newline="") as arquivo_csv:
        escritor = csv.writer(arquivo_csv)
        escritor.writerows(resultados)

In [8]:
def jogar_jogo():
    tabuleiro = criar_tabuleiro()
    jogador_atual = "X"
    partida = []

    while True:
        mostrar_tabuleiro(tabuleiro)
        try:
            posicao = int(input(f"Jogador {jogador_atual}, escolha uma posição (0-8): "))
            if tabuleiro[posicao] != " ":
                print("Posição já ocupada! Tente novamente.")
                continue
        except (ValueError, IndexError):
            print("Entrada inválida! Escolha um número de 0 a 8.")
            continue

        tabuleiro[posicao] = jogador_atual
        partida.append(jogador_atual + str(posicao))

        if verificar_vitoria(tabuleiro, jogador_atual):
            mostrar_tabuleiro(tabuleiro)
            print(f"Jogador {jogador_atual} venceu!")
            return partida, jogador_atual

        if verificar_empate(tabuleiro):
            mostrar_tabuleiro(tabuleiro)
            print("Empate!")
            return partida, "Empate"

        jogador_atual = "O" if jogador_atual == "X" else "X"

In [13]:
# @title Menu Principal do jogo
def main():
    print("Bem-vindo ao Jogo da Velha!")
    resultados = []
    while True:
        partida, vencedor = jogar_jogo()
        resultados.append(partida + [vencedor])
        jogar_novamente = input("Deseja jogar novamente? (s/n): ").strip().lower()
        if jogar_novamente != "s":
            salvar_resultados_csv(resultados)
            print("Resultados salvos em resultados.csv.")
            print("Obrigado por jogar! Até a próxima.")
            break

if _name_ == "_main_":
    main()

Bem-vindo ao Jogo da Velha!
  |   |  
--+---+--
  |   |  
--+---+--
  |   |  
Jogador X, escolha uma posição (0-8): 0
X |   |  
--+---+--
  |   |  
--+---+--
  |   |  
Jogador O, escolha uma posição (0-8): 3
X |   |  
--+---+--
O |   |  
--+---+--
  |   |  
Jogador X, escolha uma posição (0-8): 4
X |   |  
--+---+--
O | X |  
--+---+--
  |   |  
Jogador O, escolha uma posição (0-8): 8
X |   |  
--+---+--
O | X |  
--+---+--
  |   | O
Jogador X, escolha uma posição (0-8): 7
X |   |  
--+---+--
O | X |  
--+---+--
  | X | O
Jogador O, escolha uma posição (0-8): 2
X |   | O
--+---+--
O | X |  
--+---+--
  | X | O
Jogador X, escolha uma posição (0-8): 1
X | X | O
--+---+--
O | X |  
--+---+--
  | X | O
Jogador X venceu!
Deseja jogar novamente? (s/n): s
  |   |  
--+---+--
  |   |  
--+---+--
  |   |  
Jogador X, escolha uma posição (0-8): 3
  |   |  
--+---+--
X |   |  
--+---+--
  |   |  
Jogador O, escolha uma posição (0-8): 7
  |   |  
--+---+--
X |   |  
--+---+--
  | O |  
Jogador X, 



---



In [14]:
# @title Treinando a Rede Neural
import csv
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset

In [15]:
#criando dataset personalizado para manipular os dados do Jogo da Velha.
class JogoDaVelhaDataset(Dataset):
    def __init__(self, X, y):
        self.X = torch.tensor(X, dtype=torch.float32)
        self.y = torch.tensor(y, dtype=torch.long)

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

In [16]:
#A Arquitetura está configurada com 9 entradas (uma para cada posição do tabuleiro),
# duas camadas escondidas de 64 neurônios cada,
# e uma camada de saída com 3 neurônios para classificar o resultado (vitória de X, vitória de O, ou empate)
class ModeloJogoDaVelha(nn.Module):
    def __init__(self):
        super(ModeloJogoDaVelha, self).__init__()
        self.rede = nn.Sequential(
            nn.Linear(9, 64),
            nn.ReLU(),
            nn.Linear(64, 64),
            nn.ReLU(),
            nn.Linear(64, 3),
            nn.Softmax(dim=1)
        )

    def forward(self, x):
        return self.rede(x)

In [17]:
# Lê o arquivo CSV e converte cada linha em uma lista de jogadas.
def carregar_dados_csv(nome_arquivo):
    dados = []
    with open(nome_arquivo, mode="r") as arquivo_csv:
        leitor = csv.reader(arquivo_csv)
        for linha in leitor:
            dados.append(linha)
    return dados


In [18]:
#Converte o tabuleiro em um vetor de 9 posições onde:
#0 indica espaço vazio.
#1 indica que o jogador "X" jogou na posição.
#-1 indica que o jogador "O" jogou na posição.

def preprocessar_dados(dados):
    X = []  # estado do tabuleiro
    y = []  # resultado do jogo

    for partida in dados:
        tabuleiro = [0] * 9  # Representação do tabuleiro inicial (0 = vazio)
        for jogada in partida[:-1]:  # Todas as jogadas, exceto o resultado final
            jogador = 1 if jogada[0] == 'X' else -1  # 'X' = 1, 'O' = -1
            posicao = int(jogada[1])
            tabuleiro[posicao] = jogador

        resultado = partida[-1]  # Último elemento é o resultado ('X', 'O' ou 'Empate')
        if resultado == 'X':
            y.append(1)  # X venceu
        elif resultado == 'O':
            y.append(2)  # O venceu
        else:
            y.append(0)  # Empate

        X.append(tabuleiro)

    X = np.array(X)
    y = np.array(y)
    return X, y

In [19]:
def treinar_modelo(modelo, dataloader, criterio, otimizador, epochs=50):
    for epoch in range(epochs):
        modelo.train() # Modo de treino
        total_loss = 0
        for X_batch, y_batch in dataloader:
            otimizador.zero_grad()
            saida = modelo(X_batch)
            loss = criterio(saida, y_batch) # Erro quadrático médio
            loss.backward() # Calcula os gradientes dos pesos em relação à perda
            otimizador.step() # Atualizar os pesos do modelo com base no gradiente da perda
            total_loss += loss.item() #Armazena a perda
        print(f"Epoch {epoch+1}/{epochs}, Loss: {total_loss/len(dataloader):.4f}")

In [20]:
def main():
    nome_arquivo = "resultados.csv"
    dados = carregar_dados_csv(nome_arquivo)
    X, y = preprocessar_dados(dados)

    # Criar dataset e dataloader
    dataset = JogoDaVelhaDataset(X, y)
    dataloader = DataLoader(dataset, batch_size=8, shuffle=True)

    # Criar modelo que será treinado
    modelo = ModeloJogoDaVelha()
    criterio = nn.CrossEntropyLoss() #avalia quão erradas estão as previsões comparadas aos rótulos verdadeiros.
    otimizador = optim.Adam(modelo.parameters(), lr=0.001)

    treinar_modelo(modelo, dataloader, criterio, otimizador, epochs=50)

    torch.save(modelo.state_dict(), "modelo_jogo_da_velha.pth")
    print("Modelo treinado e salvo como modelo_jogo_da_velha.pth")

if __name__ == "__main__":
    main()

Epoch 1/50, Loss: 1.1088
Epoch 2/50, Loss: 1.0967
Epoch 3/50, Loss: 1.0849
Epoch 4/50, Loss: 1.0737
Epoch 5/50, Loss: 1.0625
Epoch 6/50, Loss: 1.0513
Epoch 7/50, Loss: 1.0401
Epoch 8/50, Loss: 1.0287
Epoch 9/50, Loss: 1.0172
Epoch 10/50, Loss: 1.0055
Epoch 11/50, Loss: 0.9939
Epoch 12/50, Loss: 0.9826
Epoch 13/50, Loss: 0.9710
Epoch 14/50, Loss: 0.9592
Epoch 15/50, Loss: 0.9472
Epoch 16/50, Loss: 0.9350
Epoch 17/50, Loss: 0.9225
Epoch 18/50, Loss: 0.9097
Epoch 19/50, Loss: 0.8967
Epoch 20/50, Loss: 0.8835
Epoch 21/50, Loss: 0.8700
Epoch 22/50, Loss: 0.8562
Epoch 23/50, Loss: 0.8425
Epoch 24/50, Loss: 0.8287
Epoch 25/50, Loss: 0.8149
Epoch 26/50, Loss: 0.8011
Epoch 27/50, Loss: 0.7873
Epoch 28/50, Loss: 0.7734
Epoch 29/50, Loss: 0.7597
Epoch 30/50, Loss: 0.7460
Epoch 31/50, Loss: 0.7326
Epoch 32/50, Loss: 0.7195
Epoch 33/50, Loss: 0.7068
Epoch 34/50, Loss: 0.6946
Epoch 35/50, Loss: 0.6828
Epoch 36/50, Loss: 0.6716
Epoch 37/50, Loss: 0.6609
Epoch 38/50, Loss: 0.6508
Epoch 39/50, Loss: 0.



---




**JOGANDO COM A REDE NEURAL**

In [9]:
import torch
import torch.nn as nn


In [10]:
class ModeloJogoDaVelha(nn.Module):
    def __init__(self):
        super(ModeloJogoDaVelha, self).__init__()
        self.rede = nn.Sequential(
            nn.Linear(9, 64),
            nn.ReLU(),
            nn.Linear(64, 64),
            nn.ReLU(),
            nn.Linear(64, 3),
            nn.Softmax(dim=1)
        )

    def forward(self, x):
        return self.rede(x)

In [11]:
def jogar_com_rede_neural():
    modelo = ModeloJogoDaVelha()
    modelo.load_state_dict(torch.load("modelo_jogo_da_velha.pth"))
    modelo.eval()

    def mostrar_tabuleiro(tabuleiro):
        simbolos = {0: " ", 1: "X", -1: "O"}
        print("\nTabuleiro:")
        for i in range(0, 9, 3):
            print(f" {simbolos[tabuleiro[i]]} | {simbolos[tabuleiro[i+1]]} | {simbolos[tabuleiro[i+2]]} ")
            if i < 6:
                print("---+---+---")

    def verificar_vencedor(tabuleiro):
        combinacoes = [
            [0, 1, 2], [3, 4, 5], [6, 7, 8],  # Linhas
            [0, 3, 6], [1, 4, 7], [2, 5, 8],  # Colunas
            [0, 4, 8], [2, 4, 6]             # Diagonais
        ]
        for linha in combinacoes:
            soma = tabuleiro[linha[0]] + tabuleiro[linha[1]] + tabuleiro[linha[2]]
            if soma == 3:
                return 1  # X venceu
            elif soma == -3:
                return -1  # O venceu
        if 0 not in tabuleiro:
            return 0  # Empate
        return None

    tabuleiro = [0] * 9

    while True:
        try:
            jogador_humano = int(input("Escolha seu jogador (1 para 'X', -1 para 'O'): "))
            if jogador_humano in [1, -1]:
                break
            else:
                print("Entrada inválida. Escolha 1 para 'X' ou -1 para 'O'.")
        except ValueError:
            print("Entrada inválida. Escolha 1 para 'X' ou -1 para 'O'.")

    jogador_rede = -1 if jogador_humano == 1 else 1
    print(f"Você será o jogador {'X' if jogador_humano == 1 else 'O'} ({jogador_humano}). Rede Neural será {'X' if jogador_rede == 1 else 'O'} ({jogador_rede}).")

    while True:
        mostrar_tabuleiro(tabuleiro)
        vencedor = verificar_vencedor(tabuleiro)
        if vencedor is not None:
            if vencedor == 1:
                print("'X' venceu!")
            elif vencedor == -1:
                print("'O' venceu!")
            else:
                print("Empate!")
            break

        if sum(tabuleiro) == 0 or (sum(tabuleiro) == -1 and jogador_humano == 1) or (sum(tabuleiro) == 1 and jogador_humano == -1):  # Vez do humano
            while True:
                try:
                    posicao = int(input("Escolha sua posição (0-8): "))
                    if tabuleiro[posicao] == 0:
                        tabuleiro[posicao] = jogador_humano
                        break
                    else:
                        print("Posição ocupada. Escolha outra.")
                except (ValueError, IndexError):
                    print("Entrada inválida. Escolha um número entre 0 e 8.")
        else:  # Vez da Rede Neural
            with torch.no_grad():
                entrada = torch.tensor(tabuleiro, dtype=torch.float32).unsqueeze(0)
                predicao = modelo(entrada)
                movimento = torch.argmax(predicao, dim=1).item()
                while tabuleiro[movimento] != 0:  # Garante que o movimento é válido
                    predicao[0, movimento] = -float('inf')
                    movimento = torch.argmax(predicao, dim=1).item()
                tabuleiro[movimento] = jogador_rede

In [ ]:
if __name__ == "__main__":
    jogar_com_rede_neural()

<ipython-input-11-a3dc33d9c66b>:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  modelo.load_state_dict(torch.load("modelo_jogo_da_velha.pth"))
